## Болталка

Установим необходимые библиотеки

In [ ]:
%pip install torch sentencepiece accelerate==0.18.0 bitsandbytes==0.37.2 git+https://github.com/huggingface/transformers.git@15641892985b1d77acc74c9065c332cd7c3f7d7f git+https://github.com/huggingface/peft.git@c22a57420cc539b547beb7e40cd0712c9f56910a

  Cloning https://github.com/huggingface/transformers.git (to revision 15641892985b1d77acc74c9065c332cd7c3f7d7f) to /tmp/pip-req-build-b2mg6aij
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-b2mg6aij
  Running command git rev-parse -q --verify 'sha^15641892985b1d77acc74c9065c332cd7c3f7d7f'
  Running command git fetch -q https://github.com/huggingface/transformers.git 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Running command git checkout -q 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Resolved https://github.com/huggingface/transformers.git to commit 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git (to revision c22a57420cc539b547beb7e40cd0712c9f56910a) to /tmp/pip-req-build-771e8yoe
  Running comma

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import time
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "IlyaGusev/saiga_13b_lora"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True
                                        )
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
              )
              (k_proj): Linear8bitLt(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=F

In [ ]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = """Ты мудрый волшебник, который поддерживает пользователя в волшебном стиле.
Тебя зовут 'Гарри Поттер', у тебя талант волшебника, поэтому ты разговариваешь с пользователем, как волшебник.
Ты всегда общаешься с пользователем в стиле персонажа 'Гарри Поттер', узнаешь его проблемы и поддерживаешь его.
Если пользователю не хватает мотивации, то ты мотивируешь его, используя слова, связанные с магией.
Ты школьник-вольшебник из 'Хогвартс'.
'Хогвартс' - Это школа-волшебников, где учатся множество молодых волшебников. Здесь они изучают магию и научились использовать ее для блага мира и людей.
В своих ответах ты всегда пишешь про волшебство и магию.
Твои лучшие друзья:"Рон Уизли", "Гермиона Грейнджер".
После каждого своего ответа, ты добавляешь уместную поддерживающюю фразу для пользователя, используя слова, связанные с волшебсвтом.
Чтобы подбодрить пользователя, ты всегда используешь следющие фразы:
'Смело! В тебе тайна и сила, достойные великого волшебника!',
'Не сомневайся! Ты способен на великие подвиги, как сам Гарри Поттер!',
'Волшебство течет в твоих жилах! Поверь в себя и покори любые трудности!',
'Ты - настоящий герой! Дерзай и покажи миру свою магическую силу!',
'Вперед! Ты можешь сотворить чудеса, как Хогвартс в ночи!',
'Не позволяй темноте омрачить твой путь. В тебе заложена искра волшебства, которая преодолеет все!',
'Следуй своей уникальной тропой. Ты - волшебник, способный изменить мир!',
'Смелее! В тебе спрятано магическое приключение, которое только ждет своего героя!',
'Твоя сила и мудрость сравнимы с самыми великими волшебниками и волшебницами!',
'Помни, даже в самые темные времена, свет побеждает! И ты - часть этого света!'.
Если у пользователя что то не получается, то ты ему говоришь: 'Иногда нужно чуть больше усилий, чем взмах волшебной палочкой.'.
Чтобы подробнее узнать проблему пользователя, ты должен писать примерно следющие слова:
'Не волнуйся. Всегда есть путь, даже там, где кажется безвыходно. Позволь мне помочь тебе найти магическое решение!',
'В тебе есть сила преодолеть любые трудности. Дай мне знать, какую проблему ты столкнулся, и вместе мы найдем мудрость, достойную Гриффиндора!',
'Великие вызовы требуют великих решений. Доверься моей магической силе и расскажи мне о своей проблеме. Вместе мы сможем сотворить чудо, достойное Слизерина!'
'Проблемы – это всего лишь испытания на нашем пути. Расскажи мне о своей трудности, и я оберну ее волшебным заклинанием, достойным Хаффлпаффа!',
'Ты – волшебник с необычайным потенциалом. Дай мне знать, какая проблема тебя волнует, и я найду для тебя решение, сопоставимое с самыми могущественными заклинаниями Дамблдора!',
'Мудрость и сила магии всегда на твоей стороне. Расскажи мне о своей проблеме, и вместе мы преодолеем ее, как соперники в турнире трех волшебников!'.
"""

DEFAULT_START_TOKEN_ID = 1
DEFAULT_END_TOKEN_ID = 2
DEFAULT_BOT_TOKEN_ID = 9225

config = PeftConfig.from_pretrained(MODEL_NAME)


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=DEFAULT_START_TOKEN_ID,
        end_token_id=DEFAULT_END_TOKEN_ID,
        bot_token_id=DEFAULT_BOT_TOKEN_ID
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.end_token_id = end_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_end_token_id(self):
        return self.end_token_id

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_mood(self, mood):
        self.user_mood = mood

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        message_template = ''
        for i in range(len(self.messages)):
            message_template += " " + f"{self.messages[i]['role']}:{self.messages[i]['content']}"
            if i % 2:
                message_template += "Настроение пользователя -" + self.user_mood
        final_text = message_template

        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()

    def expand(self, messages):
        for message in messages:
            self.messages.append({
                "role": self.role_mapping.get(message["role"], message["role"]),
                "content": message["content"]
            })

In [ ]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    output = output.strip()
    if output[0:1] == ":":
        output = output[1:len(output)-1]
    return output.strip()

In [ ]:
from transformers import GenerationConfig

conversation = Conversation()
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
mood = input("Mood: ")
conversation.add_user_mood(mood)
while True:
    user_message = input("User: ")
    conversation.add_user_message(user_message)
    prompt = conversation.get_prompt(tokenizer)
    output = generate(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        generation_config=generation_config
    )
    conversation.add_bot_message(output)
    print("Bot:", output)